# Loading Model
Model `ckiplab/bert-base-chinese-qa` sẽ được load từ HuggingFace cùng với các thông số cài đặt trước để chuẩn bị cho bước huấn luyện và đánh giá.

In [ ]:
!pip install -q datasets bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
import os
import torch
from datasets import DatasetDict, Dataset
from transformers import (
    AutoModelForQuestionAnswering,
    AutoModel,
    AutoTokenizer,
    BertTokenizerFast,
    BitsAndBytesConfig,
    logging
)

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
logging.set_verbosity_error()

# Load dữ liệu để đánh giá model trước và sau khi sử dụng support database

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Support Database từ file .pkl đã lưu

In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pickle
with open('/content/drive/MyDrive/Nhom_2/Nhóm 2.2/Đồ án/new_vectors.pkl', 'rb') as f:
  # Load the data from the file
  data = pickle.load(f)

## Kết nối với Pinecone

In [ ]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=userdata.get('PINECONE_API_KEY')
)

### Upload dữ liệu vào Pinecone (đã thực hiện, không cần lặp lại)

In [ ]:
# Create new index if not existed
if 'vta-mooccubex-db' not in pc.list_indexes():
    pc.create_index(
        name='vta-mooccubex-db',
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [ ]:
# Upload data
# Since data has been uploaded to Pinecone we can skip this step
from tqdm.auto import tqdm

for d in tqdm(data[53272:], total=len(data[53272:])):
    try:
        index.upsert(vectors=[d])
    except:
        continue

  0%|          | 0/74780 [00:00<?, ?it/s]

### Load dữ liệu từ Pinecone

In [ ]:
# Connect to index
index = pc.Index('vta-mooccubex-db')

### Thử nghiệm với Pinecone bằng 1 query cho trước

In [ ]:
# Example of a query
query = "储存单位有哪些"

import numpy as np

db_token = "BAAI/bge-base-zh"

db_tokenizer = AutoTokenizer.from_pretrained(db_token, trust_remote_code=True)
db_model = AutoModel.from_pretrained('BAAI/bge-base-zh')
db_model.eval()


query_embedded = db_tokenizer(query, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = db_model(**query_embedded)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
# normalize embeddings
query_vector = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/940 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

In [ ]:
# Send the query to Pinecone
xc = index.query(vector=query_vector.tolist(), top_k=5, include_metadata=True)
xc

{'matches': [{'id': 'E0C76E1BDB560A3F9C33DC5901307461-2',
              'metadata': {'text': '一个是存，'},
              'score': 0.835356116,
              'values': []},
             {'id': 'B7A5E8BB1ECEE0FE9C33DC5901307461-0',
              'metadata': {'text': '大家好 本视频介绍文件的存储'},
              'score': 0.81591773,
              'values': []},
             {'id': '7FA1DCCC871E1E3C9C33DC5901307461-1',
              'metadata': {'text': '。'},
              'score': 0.801943541,
              'values': []},
             {'id': 'C6A3B36B193749E99C33DC5901307461-0',
              'metadata': {'text': '各位同学，'},
              'score': 0.797219455,
              'values': []},
             {'id': 'D854C076DDCB927A9C33DC5901307461-1',
              'metadata': {'text': '储蓄存款就是居民个人的存款个人的存款又可以分为居民的活期储蓄存款和定期储蓄存款另外再加上一项是定期存款这个定期存款主要是指企事业单位的定期存款另外再加上其他存款这个其他存款比如说保证金存款是算作其他存款的截止到2015年10月底我们从统计的量来看中国人民银行大家如果去中国人民银行的网站每个月人民银行都会公布M0M1和M2的数据到2015年10月底的数据显示中国的M2已经超过136万亿元其中流通当中的现金流通当中的现金也就是人民银行发行的纸币和硬币在我们企业

## Load dữ liệu đánh giá

In [ ]:
!git clone https://github.com/khanh232002/VTA-dataset --quiet

import pandas as pd
from datasets import Dataset, DatasetDict
qa = pd.read_csv('VTA-dataset/vta_qa.csv')
qa

,id,question,answer,knowledge
0,24925,试探,探听对方的意思或反应,['向外探索月球 探索群星然后将镜头扭转回来 开始看到了地球和自我这就是反思性的转折 扭...
1,24926,线性试探,是计算机程序解决散列表冲突时所采取的一种策略,['在上一节 我们已经介绍了排解散列冲突的基本方法大抵说来 无非封闭定址和开放定址两类大的层...
2,24956,什么是C语言？,是一门面向过程的、抽象化的通用程序设计语言，广泛应用于底层开发。C语言能以简易的方式编译、处...,['Ritchie（理察德）开始和Thompson（汤普森）合作开发UNIX将B语言改造成C...
3,25115,二进制转十进制,要从右到左用二进制的每个数去乘以2的相应次方，小数点后则是从左往右，如果首位是0就表示正整数...,['大家好这一讲我们来学习计算机中的数制以及它们之间的转换我们人类习惯的是十进制计数而计算机...
4,25268,计算机科学技术,是一门普通高等学校本科专业，属于计算机类专业，基本修业年限为四年，授予工学或理学学士学位；2...,['not technology”因此 从教学内容设计上我们认为至少应该包含以下两大部分一是...
...,...,...,...,...
1867,136281,计算有什么一般特点？,1、运算速度快：计算机系统的运算速度可以达到每秒万亿次。 2、计算精确度高：计算机的计算 精...,['大家好这一节我们介绍计算机及计算的本质特征对于初学者来说很多教科书都在第一章中介绍计算机...
1868,136282,什么是图灵机？,指一个抽象的机器，即将人们使用纸笔进行数学运算的过程进行抽象，由一个虚拟的机器替代人类进行数学运算,['欢迎同学们来到《软件理论基础》MOOC课堂我是清华大学教师罗贵明今天介绍的内容是图灵机的...
1869,136331,什么是操作系统？,是一组主管并控制计算机操作、运用和运行硬件、软件资源和提供公共服务来组织用户交互的相互关联的...,['操作系统是用于管理和控制计算机全部软件和硬件资源方便用户使用计算机的一组程序它的运行在硬...
1870,136370,Windows中的安全配置:,在控制台树中，单击 “计算机配置 ”，单击 “Windows 设置” ，然后单击 “安全设置 ”,['本节我们介绍Windows 7的系统安全包括用户管理 操作中心 系统还原以及BitLoc...


# Thực hiện Interference với context lấy từ Pinecone

In [ ]:
responses = []

In [ ]:
from tqdm.auto import tqdm
from transformers import pipeline

question_answerer = pipeline(
  "question-answering",
  model='ckiplab/bert-base-chinese-qa',
  tokenizer='bert-base-chinese',
  device=0
)

for _, row in tqdm(qa.iterrows(), total=qa.shape[0]):
  query = row['question']
  query_embedded = db_tokenizer(query, padding=True, truncation=True, return_tensors='pt')

  # Compute token embeddings
  with torch.no_grad():
      model_output = db_model(**query_embedded)
      # Perform pooling. In this case, cls pooling.
      sentence_embeddings = model_output[0][:, 0]

  # normalize embeddings
  query_vector = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)

  # Send the query to Pinecone
  xc = index.query(vector=query_vector.tolist(), top_k=5, include_metadata=True)

  # Use the text metadata from the query as context
  contexts = ""
  for data in xc['matches'][:]:
    contexts += data['metadata']['text']
  #contexts = xc['matches'][:][0]['metadata']['text']

  # Input the question and retrieved context into the model
  responses.append(question_answerer(question=row['question'], context=contexts))

responses[0:3]

## Evaluation using ROUGE metrics

In [ ]:
# Load saved answers
final = pd.read_excel('/content/drive/MyDrive/Nhom_2/Nhóm 2.2/Đồ án/Models/ckiplab-bert-base-chinese-qa.xlsx',header=None)

In [ ]:
!git clone https://github.com/JialeGuo/py_rouge_zh --quiet
%cd py_rouge_zh
!python setup.py install --quiet

/content/py_rouge_zh/py_rouge_zh/py_rouge_zh
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, 

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import rouge_zh

evaluator = rouge_zh.Rouge(metrics=['rouge-n'],
                           max_n=2,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

scores = evaluator.get_scores(qa['answer'].tolist(), final[0].to_list())
print(scores)

{'rouge-1': {'f': 0.002383264883264883, 'p': 0.0018899755277960407, 'r': 0.004807692307692308}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}}


In [ ]:
import rouge_zh

evaluator = rouge_zh.Rouge(metrics=['rouge-l'],
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


scores = evaluator.get_scores(qa['answer'].tolist(), final[0].to_list())
print(scores)

{'rouge-l': {'f': 0.002383264883264883, 'p': 0.0018899755277960407, 'r': 0.004807692307692308}}


# Thực hiện Interference với context lấy từ Knowledge trong `vta_qa.csv`

In [ ]:
responses = []

In [ ]:
from tqdm.auto import tqdm
from transformers import pipeline

question_answerer = pipeline(
  "question-answering",
  model='ckiplab/bert-base-chinese-qa',
  tokenizer='bert-base-chinese',
  device=0
)

for _, row in tqdm(qa.iterrows(), total=qa.shape[0]):
  responses.append(question_answerer(question=row['question'], context=row['knowledge']))

responses[0:3]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  0%|          | 0/1872 [00:00<?, ?it/s]

[{'score': 8.248861718129774e-07,
  'start': 417,
  'end': 427,
  'answer': '我开始询问真诚的问题'},
 {'score': 0.0007829491514712572,
  'start': 1897,
  'end': 1910,
  'answer': '受信号传输过程中干扰的影响'},
 {'score': 0.7141466736793518, 'start': 324, 'end': 329, 'answer': '结构式语言'}]

In [ ]:
# Export saved answer to file

import csv
answers = []
for res in responses:
	answers.append(res['answer'])
final = pd.DataFrame(answers)
final.to_excel('/content/drive/MyDrive/Nhom_2/Nhóm 2.2/Đồ án/Models/ckiplab-bert-base-chinese-qa.xlsx',header=False, index=False)

## Evaluation using ROUGE metrics

In [ ]:
# Load saved answers
final = pd.read_excel('/content/drive/MyDrive/Nhom_2/Nhóm 2.2/Đồ án/Models/ckiplab-bert-base-chinese-qa.xlsx',header=None)

In [ ]:
!git clone https://github.com/JialeGuo/py_rouge_zh --quiet
%cd py_rouge_zh
!python setup.py install --quiet

/content/py_rouge_zh/py_rouge_zh/py_rouge_zh
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, 

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import rouge_zh

evaluator = rouge_zh.Rouge(metrics=['rouge-n'],
                           max_n=2,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

scores = evaluator.get_scores(qa['answer'].tolist(), final[0].to_list())
print(scores)

{'rouge-1': {'f': 0.002383264883264883, 'p': 0.0018899755277960407, 'r': 0.004807692307692308}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}}


In [ ]:
import rouge_zh

evaluator = rouge_zh.Rouge(metrics=['rouge-l'],
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


scores = evaluator.get_scores(qa['answer'].tolist(), final[0].to_list())
print(scores)

{'rouge-l': {'f': 0.002383264883264883, 'p': 0.0018899755277960407, 'r': 0.004807692307692308}}
